In [ ]:
import xgboost as xgb
import pandas as pd
#import numerapi
import sklearn.linear_model

import matplotlib.pyplot as plt
import numpy as np
from sklearn.utils import shuffle
import time

# gradient boosting for classification in scikit-learn
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import RepeatedStratifiedKFold
from matplotlib import pyplot
import sklearn.metrics as metrics
from sklearn.preprocessing import LabelEncoder 
from xgboost import XGBClassifier
from sklearn.externals import joblib
from sklearn.utils import class_weight

from datetime import date, timedelta
import os

import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, BatchNormalization, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers

In [ ]:
def shuffle_data(X_train, Y_train):
    tic = time.time()
    X_train, Y_train = shuffle(X_train, Y_train)
    toc = time.time()
    print("shuffle the data took ", toc - tic)
    return X_train, Y_train

def calculate_weights(Y_train):
    class_weights = class_weight.compute_class_weight('balanced', np.array([0, 0.25, 0.5, 0.75, 1]), Y_train)
    class_weights = dict(enumerate(class_weights))
    return class_weights

def max_accuracy(y_true, y_pred):
  """Calculates how often the max prediction matches one-hot labels."""
  retVal = 0
  
  if y_true.shape[0] != None:  
    num_correct_classified = tf.math.argmax(y_true,axis = 1) == tf.math.argmax(y_pred, axis = 1)
    num = tf.reduce_sum(tf.dtypes.cast(num_correct_classified, tf.int32), axis = -1)
    retVal = num / y_true.shape[0]
  else:
    retVal = tf.dtypes.cast(tf.math.argmax(y_true,axis = 1) == tf.math.argmax(y_pred, axis = 1), tf.int32)
  return retVal

def defineNN(n_inputFeatures):
    activation = "relu"
    regularizationConst_l1 = 0.0002
    regularizationConst_l2 = 0.0001
    size = 512
    X_input = Input(shape=(n_inputFeatures,))
    X = Dropout(0.3, input_shape = (n_inputFeatures,))(X_input)
 
    X = Dense(size, activation=activation, kernel_regularizer=regularizers.l1_l2(l1=regularizationConst_l1, l2=regularizationConst_l2), bias_regularizer=regularizers.l2(regularizationConst_l2), activity_regularizer=regularizers.l2(regularizationConst_l2))(X)
    #X = Dropout(dropoutRate, input_shape = (size,))(X)
    X = BatchNormalization(axis = -1)(X)
    X = Dense(size, activation=activation, kernel_regularizer=regularizers.l1_l2(l1=regularizationConst_l1, l2=regularizationConst_l2), bias_regularizer=regularizers.l2(regularizationConst_l2), activity_regularizer=regularizers.l2(regularizationConst_l2))(X)
    #X = Dropout(dropoutRate, input_shape = (size,))(X)
    X = BatchNormalization(axis = -1)(X)
    X = Dense(size, activation=activation, kernel_regularizer=regularizers.l1_l2(l1=regularizationConst_l1, l2=regularizationConst_l2), bias_regularizer=regularizers.l2(regularizationConst_l2), activity_regularizer=regularizers.l2(regularizationConst_l2))(X)
    #X = Dropout(dropoutRate, input_shape = (size,))(X)
    X = BatchNormalization(axis = -1)(X)
    X = Dense(5, activation="softmax")(X)
    
    model = Model(inputs = X_input, outputs = X, name='deepNN')

    return model

def oneHotEncodeData(targets):
    j=0
    Y_val = np.zeros((targets.shape[0], 5))
    for j in range(targets.shape[0]):
        if targets[j] == 0:
            Y_val[j, 0] = 1
        elif targets[j] == 0.25:
            Y_val[j, 1] = 1
        elif targets[j] == 0.5:
            Y_val[j, 2] = 1
        elif targets[j] == 0.75:
            Y_val[j, 3] = 1
        elif targets[j] == 1.0:
            Y_val[j, 4] = 1
        else:
            print("something went wrong, new class", targets[j])
    return Y_val

In [ ]:
useValidationData = True

tic = time.time()
#training_data = pd.read_csv("appendedTrainingData_test12021-01-31_2021-03-21.csv")
training_data = pd.read_csv("data/numerai_datasets_04.04.21/numerai_training_data.csv")
feature_cols = training_data.columns[training_data.columns.str.startswith('feature')]

training_data = training_data.drop_duplicates()
training_data[feature_cols] = training_data[feature_cols].astype(np.float16)
training_data.target        = training_data.target.astype(np.float16)


X_train = training_data[feature_cols].to_numpy()
Y_train = training_data.target.to_numpy()

if useValidationData:
    tournament_data = pd.read_csv("data/numerai_datasets_04.04.21/numerai_tournament_data.csv")
    tournament_data[feature_cols] = tournament_data[feature_cols].astype(np.float16)
    tournament_data.target        = tournament_data.target.astype(np.float16)
    
    

    validation_data = tournament_data.loc[tournament_data.data_type == 'validation']
    X_val = validation_data[feature_cols].reset_index().drop(['index'], axis = 1).to_numpy()
    #X_pred = tournamend_data.loc[tournament_data.data_type == ""]
    Y_val = validation_data.target.to_numpy()


toc = time.time()
print("processed the data took ", toc - tic)

In [ ]:
training_data.to_csv("data/numerai_datasets_04.04.21/numerai_training_data.csv")
tournament_data.to_csv("data/numerai_datasets_04.04.21/numerai_tournament_data.csv")

### reduce complexity of dataset by pca 

In [ ]:
from sklearn.decomposition import PCA

# This implementation uses Minka (2000) to estimate the dimension of the lower-dimensional subspace
pca = PCA(n_components='mle')

# # Alternatively, we may specify a desired % of variance to keep
# pca = PCA(0.95)

pca.fit(X_train)
pca_train = pca.transform(X_train)
pca_test = pca.transform(X_val)

### train every era separately 

In [ ]:
#find all eras in data set
eraArray = ['era1']
for era in training_data['era']:
    if eraArray[len(eraArray)-1] != era:
        eraArray.append(era)
np.random.shuffle(eraArray)
classes = np.array([0, 0.25, 0.5, 0.75, 1])

In [ ]:
useNeuralNet = False
useXgBoost   = True

Y_val = validation_data.target.to_numpy()
if useNeuralNet:
    Y_valOneHot = oneHotEncodeData(Y_val)
    modelNN     = defineNN(X_train.shape[1])
    modelNN.compile(optimizer='adam', loss='CategoricalCrossentropy', metrics=max_accuracy)
if useXgBoost:
    model = XGBClassifier(learning_rate=0.1, n_estimators = 100, max_depth=10, colsample_bytree=0.1)
    retrainModel = True

#loop over all eras to define the training data
counter = 0
for era in eraArray:
    counter += 1
    print("era ",counter)
    
    era_data = training_data.loc[training_data.era == era]
    X_trainE = era_data[feature_cols].to_numpy()
    Y_trainE = era_data.target.to_numpy()
    
    #process data for training
    
    if useNeuralNet:
        class_weights = calculate_weights(Y_trainE)
        Y_trainE      = oneHotEncodeData(Y_trainE)
        history       = modelNN.fit(X_trainE, Y_trainE, epochs = 10, batch_size = 256*128, class_weight=class_weights, validation_data=(X_val, Y_valOneHot))
        #import pdb; pdb.set_trace()
        #model.save_weights("model.h5")
        
    if useXgBoost:
        
        class_weights = class_weight.compute_class_weight('balanced', np.array([0, 0.25, 0.5, 0.75, 1]), Y_trainE)
        w_array = np.ones(Y_trainE.shape[0], dtype = 'float')
        for i, val in enumerate(Y_trainE):
            index = np.where(classes == val)
            w_array[i] = class_weights[index]
            
        evalset = [(X_trainE, Y_trainE), (X_val,Y_val)]
        
        earlyStopping = 10
        
        if retrainModel:
            model.fit(X_trainE, Y_trainE,  eval_set=evalset, sample_weight=w_array, early_stopping_rounds=earlyStopping)
            retrainModel = False
        else:
            model.fit(X_trainE, Y_trainE,  eval_set=evalset, sample_weight=w_array, early_stopping_rounds=earlyStopping, xgb_model='model_eraWise_xgBoost.model')
        model.save_model('model_eraWise_xgBoost.model')
        

        
        
    
    

In [ ]:
modelNN.save_weights("model_eraTrained_32Train_39Val.h5")

In [ ]:
y_pred_oneHot = modelNN.predict(X_val)

pred = np.argmax(y_pred_oneHot, axis = 1) / 4
print(y_pred_oneHot, pred)
    

predictions_df = validation_data["id"].to_frame()
predictions_df["pred"] = pred
predictions_df.pred.hist(bins = 10)
print(len(predictions_df.loc[predictions_df.pred != 0.5]))